# Benchmarks

This file details the benchmark environment and code run for the experiment.

In [1]:
import numpy as np
from safeWeight import compute_p,get_AL,compute_p_one,interval_2layer_L,interval_all_L
from tqdm import trange
import time
from multiprocessing.dummy import Pool as ThreadPool
from utils import savePoly,initial
x_margin = 0.001
num=1000
w_margin=0.5

## Unstable LDS (Linear dynamics system)
$$
\begin{bmatrix}
 
 \dot{x_1}
 \\\dot{x_2}
\end{bmatrix} = \begin{bmatrix}
0.3x_2+0.05u\\
0.2u\\\end{bmatrix} 
$$
with state variables $\mathbf{x}=[x_1,x_2]$. The system domain, initial set and unsafe set are given by
\begin{gather*}
\Psi=\left\{ \mathbf{x} \in \mathbb{R}^2~|~|\mathbf{x}|_\infty \leq 1.2 \right\},\\
\Theta=\left\{ \mathbf{x} \in \mathbb{R}^2~|~|\mathbf{x}|_\infty \leq 0.4 \right\},\\
X_u=\left\{ \mathbf{x} \in \mathbb{R}^2~|~|\mathbf{x}|_\infty \geq 1.2 \right\}.
\end{gather*}
$$
\begin{aligned}
p(\mathbf{x})=&~-0.1751x_1^2 - 4.5044x_1x_2 - 8.4856x_1 - 3.8831x_2^2 - 5.9148x_2 - 0.0389 \\
B(\mathbf{x})=&~530767.6485-13470.7594x_2-0.0015x_3-1468425.1260x_2^2-0.4049x_2x_3\\
&-804302.3648x_3^2\\
p_{all}(\mathbf{x})=&-0.9619x_1^2 + 1.2904x_1x_2 - 1.2039x_1 - 0.2209x_2^2 - 3.1058x_2 + 0.1090\\
B_{all}(\mathbf{x})=&~1566907.2101+616058.7634x_2+1317.2776x_3-3712292.7097x_2^2-241.7785x_2x_3\\
&-3266622.6429x_3^2
\end{aligned}
$$



In [3]:
## LDS
env,isfirst,x_margin,w_margin,u,degree,num='lds',0,0.0005,0.1,0.5,2,5000
totalP=0
validId=[]
example,poly_reg,poly_model,x_test,y_test=initial(env,isfirst,degree,num)
length = x_test.shape[0]
poly_s = savePoly(poly_model,example.n_dim,degree)
finals = []
def poly_predict(x):
  x1 =poly_reg.fit_transform(x)
  y1 =  poly_model.predict(x1) 
  return y1
def isValid(index): 
    if index%100==0:
        print(index)
    isSafe = True
    i=0
    for j in range(length//20):
    # for i in [0]:
        x_init=(x_test[i,:]).reshape(1,-1)
        if  env!='car' and(example.isSafe(x_init.reshape(2,-1))  or np.abs(y_test[i,0])>1):
            i+=1
            continue
        samplePredict = poly_predict(x_init)
        if not isfirst:
            isPass,final = interval_2layer_L(x_init.reshape(1,example.n_dim),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        else:
            isPass,final = interval_all_L(x_init.reshape(1,example.n_dim),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        i+=20
        finals.append(final)
        if not isPass:
            isSafe = False
            break
    if isSafe:
        validId.append(index)
start = time.time()
pool = ThreadPool(20)
items = list(range(num))
pool.map(isValid,items)
pool.close()
pool.join()
end = time.time()
print(env," u:",u,',have ',len(validId),' valid weight') 
if len(validId)!=0:
    if isfirst:
        totalP,maxSigma= compute_p(validId,w_margin)
    else:
        totalP,maxSigma= compute_p_one(validId,w_margin,env,'')
    if isfirst:
        print(env,"all layer")
    else:
        print(env,"one layer")
    print(env,"finalP:",totalP,"u:",u, "x_margin:",x_margin,"w_margin:",w_margin,"runtime:",end-start,"margin:",maxSigma)


0100
200
300
400
500
600700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600

2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4300
4400
450046002700
4700
4800
4900

4200


lds  u: 0.5 ,have  5000  valid weight
lds one layer
lds finalP: 0.9986114800263923 u: 0.5 x_margin: 0.001 w_margin: 0.5 runtime: 0.10390806198120117 margin: 3.5191526769475683


## Pendulum
$$
\begin{bmatrix}
 
 \dot{\theta}
 \\\dot{\omega}
\end{bmatrix} = \begin{bmatrix}
\omega\\
\frac{-3g\cdot \sin(\alpha ) }{2l} +\frac{7.5u}{m\cdot l^2} \\\end{bmatrix} 
$$
with the constants set as $g=10.0, l=1$ and $m=0.8$, and state variables $\mathbf{x}=[\theta,\omega]$. The system domain, initial set and unsafe set are given by
$$
\begin{gather*}
\Psi=\left \{ \mathbf{x} \in  \mathbb{R}^2~|~\left |\theta \right | \le \pi~and~\left |\omega  \right | \le 8  \right \}, \\
\Theta=\left \{ \mathbf{x} \in  \mathbb{R}^2~|~\left |\theta \right | \le \pi/6~and~\left |\omega  \right | \le 0.2  \right \},\\
X_u=\left \{ \mathbf{x} \in  \mathbb{R}^2 ~|~\left |\theta \right | \ge 0.9~and~\left |\omega  \right | \ge 2  \right \}.
\end{gather*}
$$
$$
\begin{aligned}
p(\mathbf{x})=&~-0.8813\theta^3 + 3.4688\theta^2\omega + 1.755\theta^2 + 1.0134\theta\omega^2 - 1.3146\theta\omega - 12.8534\theta - 1.4159\omega^3,\\
&+ 0.9918\omega^2 - 6.284\omega + 1.1191,\\
B(\mathbf{x})=&-0.1158\theta^2 - 0.0719\theta\omega + 0.0209\theta - 0.0437\omega^2 + 0.0250\omega + 0.0658,\\
p_{all}(\mathbf{x})=&-2.2353\theta^3 + 9.2215\theta^2\omega - 4.4027\theta^2 - 1.8368\theta\omega^2 + 1.5436\theta\omega - 32.4682\theta + 0.982\omega^3\\
&+ 0.9027\omega^2 - 11.4253\omega - 0.2507,\\
B_{all}(\mathbf{x})=&-0.0898\theta^2 - 0.0233\theta\omega + 0.0097\theta - 0.0157\omega^2 - 0.0022\omega + 0.0487.
\end{aligned}
$$

In [3]:
## pend
env,isfirst,x_margin,w_margin,u,degree,num='pend',0,0.0005,0.1,0.8,3,5000
totalP=0
validId=[]
example,poly_reg,poly_model,x_test,y_test=initial(env,isfirst,degree,num)
length = x_test.shape[0]
poly_s = savePoly(poly_model,example.n_dim,degree)
finals = []
def poly_predict(x):
  x1 =poly_reg.fit_transform(x)
  y1 =  poly_model.predict(x1) 
  return y1
def isValid(index): 
    if index%100==0:
        print(index)
    isSafe = True
    i=0
    for j in range(length//20):
    # for i in [0]:
        x_init=(x_test[i,:]).reshape(1,-1)
        if  env!='car' and(example.isSafe(x_init.reshape(2,-1))  or np.abs(y_test[i,0])>1):
            i+=1
            continue
        samplePredict = poly_predict(x_init)
        if not isfirst:
            isPass,final = interval_2layer_L(x_init.reshape(1,example.n_dim),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        else:
            isPass,final = interval_all_L(x_init.reshape(1,example.n_dim),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        i+=20
        finals.append(final)
        if not isPass:
            isSafe = False
            break
    if isSafe:
        validId.append(index)
start = time.time()
pool = ThreadPool(20)
items = list(range(num))
pool.map(isValid,items)
pool.close()
pool.join()
end = time.time()
print(env," u:",u,',have ',len(validId),' valid weight') 
if len(validId)!=0:
    if isfirst:
        totalP,maxSigma= compute_p(validId,w_margin)
    else:
        totalP,maxSigma= compute_p_one(validId,w_margin,env,'')
    if isfirst:
        print(env,"all layer")
    else:
        print(env,"one layer")
    print(env,"finalP:",totalP,"u:",u, "x_margin:",x_margin,"w_margin:",w_margin,"runtime:",end-start,"margin:",maxSigma)

0
1200
700
200
900
400
1100
100
600
800
300
1000
500
1900
1400
2400
1600
2100
1300
2300
1800
2500
1500
2000
2200
1700
2900
3100
2600
3600
2800
3300
3000
3500
3700
2700
3200
3400
4100
4600
3800
4300
4800
4000
4500
4200
4700
3900
4400
4900
pend  u: 0.8 ,have  2576  valid weight
pend one layer
pend finalP: 0.9541801801628007 u: 0.8 x_margin: 0.0005 w_margin: 0.1 runtime: 8991.177038192749 margin: 2.374312905220823


## Collision avoidance
$$
\begin{bmatrix}
 
 \dot{x_1}
 \\\dot{x_2}
\\\dot{y}

\end{bmatrix} = \begin{bmatrix}u\\0\\-1\end{bmatrix}
$$
with state variables $\mathbf{x}=[x_1,x_2,y]$. The system domain, initial set and unsafe set are defined as
$$
\begin{gather*}
\Psi=\left \{ \mathbf{x} \in  \mathbb{R}^3~|~\left | x_1\right | \le 2~and~ \left | x_2 \right |\le 2~and~y\in[-1,5]  \right \},\\
\Theta=\left \{ \mathbf{x} \in  \mathbb{R}^3~|~\left | x_1\right | \le 2~and~ \left | x_2 \right |\le 2~and~y= 5  \right \},\\
X_u=\left \{ \mathbf{x} \in  \mathbb{R}^3~|~\left | x_1-x_2\right | \le 1~and~y= 0  \right \}.
\end{gather*}
$$
$$
\begin{aligned}
p(\mathbf{x})=&-0.0119x_1^2 + 0.0227x_1x_2 + 0.0010x_1y + 0.3602x_1 - 0.0024x_2^2 - 0.0531x_2y \\
&- 0.1425x_2 - 0.0014y^2 + 0.0072y + 0.0851\\
B(\mathbf{x})=&0.0509x_1^2 - 0.0759x_1x_2 + 0.0090x_1y + 0.0492x_1 + 0.0198x_2^2 + 0.0532x_2y - 0.0259x_2\\
&+ 0.0403y^2 + 0.0667y - 0.1439\\
p_{all}(\mathbf{x})=&0.0421x_1^2 + 0.0977x_1x_2 - 0.0306x_1y + 0.5049x_1 + 0.0796x_2^2 - 0.1062x_2y + 0.1093x_2 \\
&+ 0.0024y^2 - 0.0259y - 0.0463\\
B_{all}(\mathbf{x})=&~0.0573x_1^2 - 0.0785x_1x_2 + 0.0164x_1y + 0.0254x_1 + 0.0261x_2^2 + 0.0588x_2y - 0.0287x_2 \\
&+ 0.0487y^2 + 0.0489y - 0.1206
\end{aligned}
$$

In [2]:
env,isfirst,x_margin,w_margin,u,degree,num='car',0,0.0005,0.1,0.5,2,2000
totalP=0
validId=[]
example,poly_reg,poly_model,x_test,y_test=initial(env,isfirst,degree,num)
length = x_test.shape[0]
poly_s = savePoly(poly_model,example.n_dim,degree)
finals = []
def poly_predict(x):
  x1 =poly_reg.fit_transform(x)
  y1 =  poly_model.predict(x1) 
  return y1
def isValid(index): 
    if index%100==0:
        print(index)
    isSafe = True
    i=0
    for j in range(length//20):
    # for i in [0]:
        x_init=(x_test[i,:]).reshape(1,-1)
        if  env!='car' and(example.isSafe(x_init.reshape(2,-1))  or np.abs(y_test[i,0])>1):
            i+=1
            continue
        samplePredict = poly_predict(x_init)
        if not isfirst:
            isPass,final = interval_2layer_L(x_init.reshape(1,example.n_dim),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        else:
            isPass,final = interval_all_L(x_init.reshape(1,example.n_dim),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        i+=20
        finals.append(final)
        if not isPass:
            isSafe = False
            break
    if isSafe:
        validId.append(index)
start = time.time()
pool = ThreadPool(20)
items = list(range(num))
pool.map(isValid,items)
pool.close()
pool.join()
end = time.time()
print(env," u:",u,',have ',len(validId),' valid weight') 
if len(validId)!=0:
    if isfirst:
        totalP,maxSigma= compute_p(validId,w_margin)
    else:
        totalP,maxSigma= compute_p_one(validId,w_margin,env,'')
    if isfirst:
        print(env,"all layer")
    else:
        print(env,"one layer")
    print(env,"finalP:",totalP,"u:",u, "x_margin:",x_margin,"w_margin:",w_margin,"runtime:",end-start,"margin:",maxSigma)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
car  u: 0.5 ,have  2000  valid weight
car one layer
car finalP: 0.9968178806423043 u: 0.5 x_margin: 0.001 w_margin: 0.5 runtime: 22.833391189575195 margin: 3.1805180100898682


## Dubin’s Car

$$
\begin{bmatrix}
 
 \dot{d}
 \\\dot{\theta}
\end{bmatrix} = \begin{bmatrix}\sin{\theta}\\-u \end{bmatrix} 
$$
with state variables $\mathbf{x}=[d,\theta]$. The system domain, initial set and unsafe set are defined as 
$$
\begin{gather}
\Psi=\left \{ \mathbf{x} \in  \mathbb{R}^2~|~ [-6, -7\pi/10]^T\le \mathbf{x}\le [6, 7\pi/10]^T \right \}, \\
\Theta=\left \{ \mathbf{x} \in  \mathbb{R}^2~|~[-1, -\pi/16]^T\le \mathbf{x}\le [1, \pi/6]^T \right\},\\
X_u=\left \{ \mathbf{x} \in  \mathbb{R}^2 ~|~[-5, -\pi/2]^T\le \mathbf{x}\le [5, \pi/2]^T  \right \}.
\end{gather}
$$
$$
\begin{aligned}
p(\mathbf{x})=&-0.3095d^2 - 2.2112d\theta + 4.8151d - 2.4769\theta^2 + 7.1023\theta + 0.0524\\
B(\mathbf{x})=&-0.094d^2 - 0.1571d\theta - 0.0048d - 0.1310\theta^2 - 0.0185\theta + 0.1384\\ 
 &+ 0.0393z^2+ 0.0763y - 0.0998
\end{aligned}
$$

In [3]:
id,isfirst,x_margin,w_margin,u,degree,sample_number='10',1,0.001,0.5,0.3,2,1000
totalP=0
validId=[]
example,poly_reg,poly_model,x_test,y_test=initial(id,isfirst,degree,num)

items = list(range(sample_number))
length = x_test.shape[0]
poly_s = savePoly(poly_model,example.n_obs,degree)
def poly_predict(x):
  x1 =poly_reg.fit_transform(x)
  y1 =  poly_model.predict(x1) 
  return y1
def isValid(index):
    isSafe = True
    if index%100==0:
        print(index)
    for i in range(length//200):
    # for i in [0]:
        x_init=(x_test[i,:]).reshape(1,-1)
        samplePredict = poly_predict(x_init)
        isPass,diff,l_poly,l,final = get_AL(x_init.reshape(1,example.n_obs),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        i+=200
        # finals.append(final)
        if not isPass:
            isSafe = False
            break

    if isSafe:
        validId.append(index)
start = time.time()
print(time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime()))
pool = ThreadPool(20)
pool.map(isValid,items)
pool.close()
pool.join()
end = time.time()
print(str(id)," u:",u,',have ',len(validId),'valid weight') 
if len(validId)!=0:
    totalP,maxSigma= compute_p(validId,w_margin,'')
    print(str(id),"finalP:",totalP,"u:",u, "x_margin:",x_margin,"w_margin:",w_margin,"sigma:",maxSigma)

2022-05-18-17_59_57
0
200
100
300
500
400
600
700
900
800
10  u: 0.3 ,have  1000 valid weight
10 finalP: 0.9754101450994895 u: 0.3 x_margin: 0.001 w_margin: 0.5 sigma: 2.99993007207813


## Oscillator
$$
\begin{bmatrix}
 
 \dot{x}
 \\\dot{y}
\end{bmatrix} = \begin{bmatrix}
y\\
\gamma(1-x^2)y-x+u \end{bmatrix} 
$$
with the constants set as $\gamma=1$ and state variables $\mathbf{x}=[x, y]$. The system domain, initial set and unsafe set are defined as 
$$
\begin{gather}
\Psi=\left \{ \mathbf{x} \in  \mathbb{R}^2~|~ [-2, -2]^T\le \mathbf{x}\le [2, 2]^T \right \}, \\
\Theta=\left \{ \mathbf{x} \in  \mathbb{R}^2~|~[-0.51, 0.49]^T\le \mathbf{x}\le [-0.49, 0.51]^T \right\},\\
X_u=\left \{ \mathbf{x} \in  \mathbb{R}^2 ~|~[-0.4, 0.2]^T\le \mathbf{x}\le [0.1, 0.35]^T  \right \}.
\end{gather}
$$
$$
\begin{aligned}
p(\mathbf{x})=&0.1194x^2 - 0.0356xy + 0.8706x - 0.0150y^2 - 6.6555y + 0.2979\\
B(\mathbf{x})=&~0.1283x^2 - 0.1983xy + 0.0773x - 0.0333y^2 - 0.0904y + 0.0139
\end{aligned}
$$

In [4]:
id,isfirst,x_margin,w_margin,u,degree,sample_number='9',1,0.001,0.5,0.1,2,1000
totalP=0
validId=[]
example,poly_reg,poly_model,x_test,y_test=initial(id,isfirst,degree,num)

items = list(range(sample_number))
length = x_test.shape[0]
poly_s = savePoly(poly_model,example.n_obs,degree)
def poly_predict(x):
  x1 =poly_reg.fit_transform(x)
  y1 =  poly_model.predict(x1) 
  return y1
def isValid(index):
    isSafe = True
    if index%100==0:
        print(index)
    for i in range(length//2000):
    # for i in [0]:
        x_init=(x_test[i,:]).reshape(1,-1)
        samplePredict = poly_predict(x_init)
        isPass,diff,l_poly,l,final = get_AL(x_init.reshape(1,example.n_obs),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        i+=200
        # finals.append(final)
        if not isPass:
            isSafe = False
            break

    if isSafe:
        validId.append(index)
start = time.time()
print(time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime()))
pool = ThreadPool(20)
pool.map(isValid,items)
pool.close()
pool.join()
end = time.time()
print(str(id)," u:",u,',have ',len(validId),'valid weight') 
if len(validId)!=0:
    totalP,maxSigma= compute_p(validId,w_margin,'')
    print(str(id),"finalP:",totalP,"u:",u, "x_margin:",x_margin,"w_margin:",w_margin,"sigma:",maxSigma)

2022-05-18-18_01_13
0
200
100
300
500
400
600
700
900
800
9  u: 0.1 ,have  998 valid weight
9 finalP: 0.9779656681290183 u: 0.1 x_margin: 0.001 w_margin: 0.5 sigma: 3.055049474234848


## Academic 3D
$$
\begin{bmatrix}
 
 \dot{x}
 \\\dot{y}\\
 \dot{z}
\end{bmatrix} = \begin{bmatrix}
z+8y\\
-y+z\\
-z-x^2+u\end{bmatrix} 
$$
with state variables $\mathbf{x}=[x,y,z]$. The system domain, initial set and unsafe set are defined as 
$$
\begin{gather*}
\Psi=\left \{ \mathbf{x} \in  \mathbb{R}^3~|~ [-5, -5,-5]^T\le \mathbf{x}\le [5, 5,5]^T \right \}, \\
\Theta=\left \{ \mathbf{x} \in  \mathbb{R}^3~|~\left \|  \mathbf{x}-[-0.75,-1,-0.4]^T\right \|  \le 0.35 \right\},\\
X_u=\left \{ \mathbf{x} \in  \mathbb{R}^3 ~|~\left \|  \mathbf{x}-[-0.3,-0.36,0.2]^T\right \|  \le 0.30  \right \}.
\end{gather*}
$$
$$
\begin{aligned}
p(\mathbf{x})=&~1.9252x^2 + 1.3410xy + 2.3829xz - 3.3200x - 1.5971y^2 - 9.6709yz - 5.6955y - 12.4129z^2 \\
&-10.4166z + 0.1471 \\
&- 4.6882x_2- 3.9432x_3^2 - 15.2565x_3 + 0.0332\\
B(\mathbf{x})=&~-0.0018x^4 + 0.0188x^3y + 0.008x^3z - 0.0138x^3 - 0.0071x^2y^2 + 0.0425x^2yz - 0.0088x^2y\\
&+ 0.0409x^2z^2 - 0.0533x^2z + 0.0330x^2 + 0.0189xy^3 + 0.0394xy^2z + 0.0007xy^2 + 0.0586xyz^2 \\
&+ 0.0116xyz - 0.0291xy + 0.0252xz^3 - 0.0488xz^2 + 0.0109xz + 0.0144x + 0.0039y^4 \\
&- 0.0044y^3z - 0.0060y^3 + 0.0023y^2z^2 - 0.0043y^2z + 0.0162y^2 + 0.0373yz^3 + 0.0350yz^2 \\
&+ 0.0153yz + 0.0142y + 0.0057z^4 - 0.0230z^3 - 0.0213z^2 + 0.0191z - 0.0008
\end{aligned}
$$

In [2]:
id,isfirst,x_margin,w_margin,u,degree,sample_number='7',1,0.001,0.5,1.6,2,1000
totalP=0
validId=[]
example,poly_reg,poly_model,x_test,y_test=initial(id,isfirst,degree,num)

items = list(range(sample_number))
length = x_test.shape[0]
poly_s = savePoly(poly_model,example.n_obs,degree)
def poly_predict(x):
  x1 =poly_reg.fit_transform(x)
  y1 =  poly_model.predict(x1) 
  return y1
def isValid(index):
    isSafe = True
    if index%100==0:
        print(index)
    for i in range(length//200):
    # for i in [0]:
        x_init=(x_test[i,:]).reshape(1,-1)
        samplePredict = poly_predict(x_init)
        isPass,diff,l_poly,l,final = get_AL(x_init.reshape(1,example.n_obs),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        i+=200
        # finals.append(final)
        if not isPass:
            isSafe = False
            break

    if isSafe:
        validId.append(index)
start = time.time()
print(time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime()))
pool = ThreadPool(20)
pool.map(isValid,items)
pool.close()
pool.join()
end = time.time()
print(str(id)," u:",u,',have ',len(validId),'valid weight') 
if len(validId)!=0:
    totalP,maxSigma= compute_p(validId,w_margin,'')
    print(str(id),"finalP:",totalP,"u:",u, "x_margin:",x_margin,"w_margin:",w_margin,"sigma:",maxSigma)

2022-05-23-13_05_11
0
200
100
300
500
400
600
700
900
800
7  u: 1.6 ,have  1000 valid weight
7 finalP: 0.9727450072449557 u: 1.6 x_margin: 0.001 w_margin: 0.5 sigma: 2.984848162378007


## Bicycle Steering
$$
\left[\begin{array}{c}
\dot{x}_{1} \\
\dot{x_{2}} \\
\dot{x}_{3}
\end{array}\right]=\left[\begin{array}{c}
x_{2} \\
\frac{m \ell}{J}\left(g \sin \left(x_{1}\right)+\frac{v^{2}}{b} \cos \left(x_{1}\right) \tan \left(x_{3}\right)\right) \\
0
\end{array}\right]+\left[\begin{array}{c}
0 \\
\frac{a m \ell v}{J b} \frac{\cos \left(x_{1}\right)}{\cos ^{2}\left(x_{3}\right)} \\
1
\end{array}\right] u
$$
with the constants set as $m=20, b=1, \ell=1, J=\frac{1}{3}mb^2, v=10, a=0.5$ and $g=10$, and state variables $\mathbf{x}=[x,y,z]$. The system domain, initial set and unsafe set are defined as 
$$
\begin{gather*}
\Psi=\left \{ \mathbf{x} \in  \mathbb{R}^3~|~ [-2.2, -2.2,-2.2]^T\le \mathbf{x}\le [2.2, 2.2,2.2]^T \right \}, \\
\Theta=\left \{ \mathbf{x} \in  \mathbb{R}^3~|~ [-0.2, -0.2,-0.2]^T\le \mathbf{x}\le [0.2, 0.2,0.2]^T \right \}, \\
X_u=\left \{ \mathbf{x} \in  \mathbb{R}^3~|~ [-2.2, -2.2,-2.2]^T\le \mathbf{x}\le [-2, -2,-2]^T \right \}.
\end{gather*}
$$
$$
\begin{aligned}
p(\mathbf{x})=&-2.0968x_1^2 - 7.1743x_1x_2 + 3.4830x_1x_3 - 7.1534x_1 - 4.0626x_2^2 + 5.9221x_2x_3 \\
&- 4.6882x_2- 3.9432x_3^2 - 15.2565x_3 + 0.0332\\
B(\mathbf{x})=&~0.0896x_1^2 - 0.1627x_1x_2 - 0.0423x_1x_3 + 0.0315x_1 + 0.0535x_2^2 + 0.0734x_2x_3\\
&+ 0.0407x_2 + 0.0698x_3^2 + 0.1364x_3 + 0.0443
\end{aligned}
$$

In [9]:
id,isfirst,x_margin,w_margin,u,degree,sample_number='12',1,0.0005,0.1,0.7,2,5000
totalP=0
validId=[]
example,poly_reg,poly_model,x_test,y_test=initial(id,isfirst,degree,sample_number)

items = list(range(sample_number))
length = x_test.shape[0]
poly_s = savePoly(poly_model,example.n_obs,degree)
def poly_predict(x):
  x1 =poly_reg.fit_transform(x)
  y1 =  poly_model.predict(x1) 
  return y1
def isValid(index):
    isSafe = True
    if index%100==0:
        print(index)
    for i in range(length//200):
    # for i in [0]:
        x_init=(x_test[i,:]).reshape(1,-1)
        samplePredict = poly_predict(x_init)
        isPass,diff,l_poly,l,final = get_AL(x_init.reshape(1,example.n_obs),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        i+=200
        # finals.append(final)
        if not isPass:
            isSafe = False
            break

    if isSafe:
        validId.append(index)
start = time.time()
print(time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime()))
pool = ThreadPool(20)
pool.map(isValid,items)
pool.close()
pool.join()
end = time.time()
print(str(id)," u:",u,',have ',len(validId),'valid weight') 
if len(validId)!=0:
    totalP,maxSigma= compute_p(validId,w_margin,'')
    print(str(id),"finalP:",totalP,"u:",u, "x_margin:",x_margin,"w_margin:",w_margin,"sigma:",maxSigma)

2022-05-22-19_37_48
0
1200
700
200
900
400
1100
600
100
800
300
1000
500
2400
1900
1400
2100
1600
2300
1800
1300
2500
2000
1500
2200
1700
2900
3600
3100
2600
3300
2800
3500
3000
3700
3200
2700
3400
4600
4100
4800
4300
3800
4500
4000
4700
4200
4900
4400
3900
12  u: 0.7 ,have  5000 valid weight
12 finalP: 0.9554378468792399 u: 0.7 x_margin: 0.0005 w_margin: 0.1 sigma: 2.966640535237153


## Chesi 3
$$
\left[\begin{array}{c}
\dot{x_{1}} \\
\dot{x_{2}} \\
\dot{x_{3}} \\
\dot{x_{4}}
\end{array}\right]=\left[\begin{array}{l}
-x_{1}-x_{4}+u \\
x_{1}-x_{2}+x_{1}^{2}+u \\
-x_{3}+x_{4}+x_{2}^{2} \\
x_{1}-x_{2}-x_{4}+x_{3}^{3}-x_{4}^{3}
\end{array}\right]
$$
with state variables $\mathbf{x}=[x_1,x_2,x_3,x_4]$.
$$
\begin{gather*}
\Psi=\left \{ \mathbf{x} \in  \mathbb{R}^4~|~ \left \|  \mathbf{x}\right \|  \le 16 \right \}, \\
\Theta=\left \{ \mathbf{x} \in  \mathbb{R}^4~|~ [-0.2, -0.2,-0.2,-0.2]^T\le \mathbf{x}\le [0.2, 0.2,0.2,0.2]^T \right \}, \\
X_u=\left \{ \mathbf{x} \in  \mathbb{R}^4~|~ \left \|  \mathbf{x} -[2,2,2,2]^T\right \|  \le 1 \right \}.
\end{gather*}
$$
$$
\begin{aligned}
p(\mathbf{x})=&~0.1929x_1^2 + 0.7244x_1x_2 - 1.0765x_1x_3 + 0.4606x_1x_4 + 0.2695x_1 - 0.1219x_2^2 \\
&- 0.1762x_2x_3 + 0.3806x_2x_4 + 0.2842x_2 + 0.4491x_3^2 - 0.9554x_3x_4 - 0.1810x_3 \\
&+ 0.1767x_4^2 + 0.0464x_4 + 0.4008 \\
B(\mathbf{x})=&-0.0586x_1^2 + 0.0118x_1x_2 + 0.0157x_1x_3 + 0.1143x_1x_4 + 0.0330x_1 - 0.0887x_2^2 \\
&+ 0.0074x_2x_3 + 0.0208x_2x_4 + 0.0586x_2 - 0.0140x_3^2 + 0.0020x_3x_4 + 0.0231x_3\\
&- 0.1094x_4^2 - 0.0815x_4 + 0.1981
\end{aligned}
$$

In [4]:
id,isfirst,x_margin,w_margin,u,degree,sample_number='20',1,0.001,0.5,0.1,2,1000
totalP=0
validId=[]
example,poly_reg,poly_model,x_test,y_test=initial(id,isfirst,degree,num)

items = list(range(sample_number))
length = x_test.shape[0]
poly_s = savePoly(poly_model,example.n_obs,degree)
def poly_predict(x):
  x1 =poly_reg.fit_transform(x)
  y1 =  poly_model.predict(x1) 
  return y1
def isValid(index):
    isSafe = True
    if index%100==0:
        print(index)
    for i in range(length//200):
    # for i in [0]:
        x_init=(x_test[i,:]).reshape(1,-1)
        samplePredict = poly_predict(x_init)
        isPass,diff,l_poly,l,final = get_AL(x_init.reshape(1,example.n_obs),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        i+=200
        # finals.append(final)
        if not isPass:
            isSafe = False
            break

    if isSafe:
        validId.append(index)
start = time.time()
print(time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime()))
pool = ThreadPool(20)
pool.map(isValid,items)
pool.close()
pool.join()
end = time.time()
print(str(id)," u:",u,',have ',len(validId),'valid weight') 
if len(validId)!=0:
    totalP,maxSigma= compute_p(validId,w_margin,'')
    print(str(id),"finalP:",totalP,"u:",u, "x_margin:",x_margin,"w_margin:",w_margin,"sigma:",maxSigma)

2022-05-22-16_37_13
0
200
100
300
500
400
600
700
900
800
20  u: 0.1 ,have  998 valid weight
20 finalP: 0.9675632607990863 u: 0.1 x_margin: 0.001 w_margin: 0.5 sigma: 3.0599367809398985


## LALO20

$$
\left[\begin{array}{c}
\dot{x_{1}} \\
\dot{x_{2}} \\
\dot{x_{3}} \\
\dot{x_{4}} \\
\dot{x_{5}} \\
\dot{x_{6}} \\
\dot{x_{7}}
\end{array}\right]=\left[\begin{array}{c}
1.4 x_{3}-0.9 x_{1} \\
2.5 x_{5}-1.5 x_{2}+u \\
0.6 x_{7}-0.8 x_{2} x_{3} \\
2-1.3 x_{3} x_{4} \\
0.7 x_{1}-x_{4} x_{5} \\
0.3 x_{1}-3.1 x_{6} \\
1.8 x_{6}-1.5 x_{2} x_{7}
\end{array}\right]
$$
with state variables $\mathbf{x}=\left[x_{1}, x_{2}, x_{3}, x_{4}, x_{5}, x_{6}, x_{7}\right]$.
$$
\begin{gather}
\Psi=\left \{ \mathbf{x} \in  \mathbb{R}^7~|~ \mathbf{a_1}\le \mathbf{x}\le \mathbf{a_2} \right \}, \\
\Theta=\left \{ \mathbf{x} \in  \mathbb{R}^7~|~ \mathbf{b_1}\le \mathbf{x}\le \mathbf{b_2} \right \}, \\
X_u=\left \{ \mathbf{x} \in  \mathbb{R}^7~|~ \mathbf{c_1}\le \mathbf{x}\le \mathbf{c_2} \right \},
\end{gather}
$$
where $\mathbf{a_1}=[-3.8, -3.95,-3.50,-2.60,-4.00,-4.90,-4.55]^T,\mathbf{a_2}=[6.20,6.05,6.50,7.40,6.00,5.10,5.45]^T,\mathbf{b_1}=[1.15,1.00,1.45,2.35,0.95,0.05,0.40]^T,\mathbf{b_2}=[1.25,1.10,1.55,2.45,1.05,0.15,0.50]^T,\mathbf{c_1}=[-3.30,-3.45,-3.00,-2.10,-3.50,-4.40,-4.05]^T$, and $\mathbf{c_1}=[1.25,1.10,1.55,2.45,1.05,1.05,0.50]^T$
$$
\begin{aligned}
p(\mathbf{x})=&~0.3632x_1^2 + 0.4139x_1x_2 - 0.2586x_1x_3 + 0.2487x_1x_4 - 0.6496x_1x_5 - 0.5303x_1x_6\\
&+ 1.8798x_1x_7 - 1.6597x_1 + 0.0316x_2^2 - 0.1971x_1x_3 + 0.2116x_1x_4 + 0.1894x_1x_5\\
&- 0.7529x_1x_6 + 0.1739x_1x_7 - 0.7287x_1 - 0.0041x_3^2 - 0.1067x_3x_4 + 0.0231x_3x_5 \\
&- 0.231x_3x_6- 0.146x_3x_7 + 0.8764x_3 + 0.06x_4^2 - 0.3485x_4x_5 + 0.0728x_4x_6 \\
&+ 0.8098x_4x_7 - 0.6889x_4 + 0.055x_5^2 - 0.957x_5x_6 + 0.1202x_5x_7 + 1.4234x_5\\
&- 0.2866x_6^2 + 0.4249x_6x_7 + 2.4245x_6 + 0.1185x_7^2 - 4.5185x_7 + 1.619 \\
B(\mathbf{x})=&-0.0409x_1^2 - 0.0409x_1x_2 - 0.0152x_1x_3 + 0.0282x_1x_4 - 0.0038x_1x_5 - 0.0104x_1x_6\\
&+ 0.0281x_1x_7 - 0.0289x_1 - 0.04x_2^2 - 0.0086x_2x_3 + 0.0045x_2x_4 + 0.006x_2x_5 - 0.0184x_2x_6\\
&+ 0.0197x_2x_7 - 0.0176x_2 - 0.0314x_3^2 + 0.0211x_3x_4 - 0.0284x_3x_5 + 0.0146x_3x_6 - 0.0138x_3x_7\\
&+ 0.01x_3 - 0.0188x_4^2 + 0.0217x_4x_5 - 0.0037x_4x_6 + 0.0152x_4x_7 - 0.0158x_4 - 0.0294x_5^2\\
&- 0.0104x_5x_6 - 0.0449x_5x_7 + 0.0107x_5 - 0.0202x_6^2 + 0.0255x_6x_7 + 0.0099x_6 - 0.0517x_7^2\\
&+ 0.0101x_7 + 0.3105
\end{aligned}
$$

In [2]:
id,isfirst,x_margin,w_margin,u,degree,sample_number='13',1,0.001,0.5,0.1,2,1000
totalP=0
validId=[]
example,poly_reg,poly_model,x_test,y_test=initial(id,isfirst,degree,num)

items = list(range(sample_number))
length = x_test.shape[0]
poly_s = savePoly(poly_model,example.n_obs,degree)
def poly_predict(x):
  x1 =poly_reg.fit_transform(x)
  y1 =  poly_model.predict(x1) 
  return y1
def isValid(index):
    isSafe = True
    if index%100==0:
        print(index)
    for i in range(length//200):
    # for i in [0]:
        x_init=(x_test[i,:]).reshape(1,-1)
        samplePredict = poly_predict(x_init)
        isPass,diff,l_poly,l,final = get_AL(x_init.reshape(1,example.n_obs),x_margin,samplePredict.reshape(1),u,w_margin,index,poly_s)
        i+=200
        # finals.append(final)
        if not isPass:
            isSafe = False
            break

    if isSafe:
        validId.append(index)
start = time.time()
print(time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime()))
pool = ThreadPool(20)
pool.map(isValid,items)
pool.close()
pool.join()
end = time.time()
print(str(id)," u:",u,',have ',len(validId),'valid weight') 
if len(validId)!=0:
    totalP,maxSigma= compute_p(validId,w_margin,'')
    print(str(id),"finalP:",totalP,"u:",u, "x_margin:",x_margin,"w_margin:",w_margin,"sigma:",maxSigma)

2022-05-22-13_55_45
0
200
100
300
500
400
600
700
900
800
13  u: 0.1 ,have  995 valid weight
13 finalP: 0.953276069420472 u: 0.1 x_margin: 0.001 w_margin: 0.5 sigma: 2.9880229559858744
